In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import os
import imageio
from tqdm.notebook import tqdm
import time
import numba as nb
from scipy import integrate
plt.ion()
%matplotlib notebook

In [2]:
def get_kernel(n,r0):
    x=np.fft.fftfreq(n)*n
    rsqr=np.outer(np.ones(n),x**2)
    rsqr=rsqr+rsqr.T
    rsqr[rsqr<r0**2]=r0**2
    kernel=rsqr**-0.5
    return kernel

@nb.njit(parallel = True) 
def get_grid_idx(pos,grid_sidelength):
    grididxs = np.zeros((len(pos),2))
    if np.ptp(pos[:,0])==0:
        xptp = 1
    else:
        xptp = np.ptp(pos[:,0])
    if np.ptp(pos[:,1])==0:
        yptp = 1
    else:
        yptp = np.ptp(pos[:,1])
        
    for i in nb.prange(len(pos)):
        xy = pos[i]
        xgrid = grid_sidelength*(xy[0]-np.min(pos[:,0]))/xptp
        if xgrid == grid_sidelength:
            xgrid-=1
        ygrid = grid_sidelength*(xy[1]-np.min(pos[:,1]))/yptp
        if ygrid == grid_sidelength:
            ygrid-=1
        grididxs[i] = [xgrid,ygrid]
    return grididxs

def loop_coords(xy,ngrid):
    xy[xy<-0.5]=xy[xy<-0.5]+ngrid
    xy[xy>=ngrid-0.5]=xy[xy>=ngrid-0.5]-ngrid
    
def remove_outside_m(xy,m,ngrid):
    for i in range(xy.shape[1]):
        m[xy[:,i]<-0.5]=0
        m[xy[:,i]>=ngrid-0.5]=0
        
@nb.njit        
def get_hist(xy,rho,masses):
    rho[:]=0
    for  i in range(len(xy)):
        ix=int(xy[i,0]+0.5)
        iy=int(xy[i,1]+0.5)
        if masses[i]>0: #we can set m=0 to flag particles
            rho[ix,iy]+=masses[i]

@nb.njit(parallel = True)        
def get_grad(pot,gridpos,ngrid,grad,masses,rk4 = False):
    for  i in nb.prange(len(gridpos)):
        if rk4:
            ix,iy = gridpos[i,0]%ngrid,gridpos[i,1]%ngrid
        else:
            ix,iy = gridpos[i,0],gridpos[i,1]
        if ix<0:
            ix1 = ngrid-1
            ix2 = 0
            dx = ix+1
        else:
            ix1 = int(ix)
            ix2 = ix1+1
            dx = ix-ix1
            if ix2==ngrid:
                ix2=0
                
        if iy<0:
            iy1 = ngrid-1
            iy2 = 0
            dy = iy+1
        else:
            iy1 = int(iy)
            iy2 = iy1+1
            dy = iy-iy1
            if iy2==ngrid:
                iy2=0

        if masses[i]>0:        
            grad[i][0] = (pot[ix2,iy2]-pot[ix1,iy2])*dy+(pot[ix2,iy1]-pot[ix1,iy1])*(1-dy)
            grad[i][1] = (pot[ix1,iy2]-pot[ix1,iy1])*(1-dx)+(pot[ix2,iy2]-pot[ix2,iy1])*dx


class Particles:
    def __init__(self,npart = 40000,n = 100,ndim = 2,periodic = True,soft = 1):
        self.npart = npart
        self.x = np.empty([npart,ndim])
        self.gridx = np.empty([npart,ndim])
        self.v = np.empty([npart,ndim])
        self.grad = np.zeros([npart,ndim])
        self.f = np.empty([npart,ndim])
        self.ngrid=n
        self.masses = np.ones(self.npart)
        self.soft = soft
        self.periodic = periodic
        self.kernel = None
        self.kernelft = None
        self.timer = False
        self.time_pot = []
        self.time_forces = []
        self.time_plot = []
        self.kinE = []
        self.potE = []
        
        if periodic:
            self.rho=np.zeros([self.ngrid,self.ngrid])
            self.pot=np.empty([self.ngrid,self.ngrid])
        else:
            self.rho=np.zeros([2*self.ngrid,2*self.ngrid])
            self.pot=np.zeros([2*self.ngrid,2*self.ngrid])
            
    def convert_to_gridcoords(self):
        self.x = get_grid_idx(self.x,self.ngrid)
        
    def get_rho(self):
        if self.periodic:
            loop_coords(self.x,self.ngrid)
        else:
            remove_outside_m(self.x,self.masses,self.ngrid)
        get_hist(self.x,self.rho,self.masses)
        
    
    def IC_Gauss(self,n_blobs = 1,center_loc = [[0,0]],stds=[1,1]):
        if n_blobs!=len(center_loc) or n_blobs!=len(center_loc):
            print('Need to specify center and std of each blob')
            
        to_remove = self.npart%n_blobs
        self.npart-=to_remove
        parts_per_blob = self.npart//n_blobs
        
        for i in range(n_blobs):
            self.x[i*parts_per_blob:(i+1)*parts_per_blob] = np.random.normal(center_loc[i],stds[i],np.shape(self.x[:parts_per_blob]))

        
    def IC_box(self,xymin = [-1,-1],xymax = [1,1]):
        self.x = np.random.uniform(xymin,xymax,size = np.shape(self.x))
        
    def get_kernel(self):
        if self.periodic:
            self.kernel=get_kernel(self.ngrid,self.soft)
        else:
            self.kernel=get_kernel(2*self.ngrid,self.soft)
        self.kernelft=np.fft.rfft2(self.kernel)
        
    def get_potential(self):
        if self.timer:
            t1 = time.time()
        self.get_rho()
        self.get_kernel()
        rhoft = np.fft.rfft2(self.rho)
        conv = np.fft.irfft2(self.kernelft*rhoft)
        self.pot = conv
        t2 = time.time()
        if self.timer:
            t2 = time.time()
            self.time_pot.append(t2-t1)
    
    def get_forces(self,rk4 = False,x=None):
        if self.timer:
            t1 = time.time()            
            
        if rk4:
            get_grad(self.pot,x,self.ngrid,self.grad,self.masses,rk4 = rk4)
            return self.grad
        
        get_grad(self.pot,self.x,self.ngrid,self.grad,self.masses)
        self.f = self.grad
        if self.timer:
            t2 = time.time()
            self.time_forces.append(t2-t1)
            
    def take_step(self,dt=1):
        self.x[:]=self.x[:]+dt*self.v
        self.get_potential()
        self.get_forces()
        self.v[:]=self.v[:]+self.f*dt
    
    def get_derivs(self,xv):
        nn=len(self.x)
        x=xv[:nn,:]
        v=xv[nn:,:]
        self.get_potential()
        f=self.get_forces(rk4 = True, x = x)
        return np.vstack([v,f])
        
    def take_step_rk4(self,dt = 1):
        
        xv = np.vstack([self.x,self.v])
        k1 = self.get_derivs(xv)
        k2 = self.get_derivs(xv+k1*dt/2)
        k3 = self.get_derivs(xv+k2*dt/2)
        k4 = self.get_derivs(xv+k3*dt)
        
        tot = (k1+2*k2+2*k3+k4)/6
        
        self.x = self.x+dt*tot[:len(tot)//2]
        self.v = self.v+dt*tot[len(tot)//2:]
        
        

    def plot_particles(self,density=False,savepath = ''):
        if self.timer:
            t1 = time.time()
        if density:
            plt.figure()
            plt.imshow(np.rot90(self.rho[:self.ngrid,:self.ngrid]),cmap = 'plasma')
        else:
            plt.figure()
            plt.plot(self.x[:,0],self.x[:,1],'.')
        if savepath !='':
            plt.savefig(savepath)
        if self.timer:
            t2 = time.time()
            self.time_plot.append(t2-t1)
            
    def plot_potential(self):
        plt.figure()
        self.get_potential()
        plt.imshow(np.rot90(particles.pot[:self.ngrid,:self.ngrid]**0.5),cmap = 'plasma')
            
    def timing_report(self):
        avg_pot_time = np.mean(self.time_pot)
        avg_f_time = np.mean(self.time_forces)
        avg_plot_time = np.mean(self.time_plot)
        
        print('With {} particles and grid sidelength {}\n'.format(self.npart,self.ngrid))
        print('Average time to compute potential: {} s'.format(round(avg_pot_time,5)))
        print('Average time to compute forces: {} s'.format(round(avg_f_time,5)))
        print('Average time to plot: {} s'.format(round(avg_plot_time,5)))
        
    def update_removing(self):
        self.masses = np.ones(len(self.x))
        self.v = np.zeros((len(self.x),2))
        self.f = np.zeros((len(self.x),2))
    
    def compute_E(self):
        self.kinE.append(np.sum(1/2*self.masses*np.sum(self.v**2)))
        self.potE.append(np.sum(self.rho*self.pot))
        
    

In [3]:
def make_animation(particles,niter,dt = 0.08,solver = 'leapfrog'):
    plt.figure()
    plt.ioff()
    path = 'Figures/Animation_{}_particles_{}_iter/'.format(particles.npart,niter)
    try:
        os.mkdir(path)
    except:
        None
        
    particles.plot_particles(density = True,savepath =path+'%04d'%0 )
    particles.compute_E()

    for i in tqdm(range(1,niter)):
        plt.clf()
        if solver == 'leapfrog':
            particles.take_step(dt = dt)
        if solver == 'rk4':
            particles.take_step_rk4(dt = dt)
        particles.plot_particles(density = True,savepath = path+'%04d'%i)
        particles.compute_E()
        plt.close()
    images = []
    filenames = os.listdir(path)
    for filename in filenames:
        images.append(imageio.imread(path+filename))
    imageio.mimsave(path+'{}_particles_{}_iter.gif'.format(particles.npart,niter), images,fps = 15)
    plt.ion()

In [ ]:
particles = Particles(npart = 1,n=70,periodic = False)
particles.v = np.zeros(np.shape(particles.v))
particles.x = np.array([[35,35]])
# todel = np.where(np.sum(particles.x**2,axis = 1)<0.5)
# particles.x = np.delete(particles.x,todel,axis = 0)
# todel = np.where(np.sum(particles.x**2,axis = 1)>4)
# particles.x = np.delete(particles.x,todel,axis = 0)
# particles.update_removing()
# particles.IC_Gauss()
# particles.x = np.array([[1,0],[-1,0]])

In [ ]:
# particles.IC_Gauss(5,np.random.randint(-5,5,size = (5,2)),[0.5,]*5)
# particles.x = np.random.uniform([-1,-1],[1,1],size = np.shape(particles.x))
# particles.IC_box(bounds = [[-2,2],[-2,2]])
# particles.x[len(particles.x)//2:] = np.random.normal([5,5],0.01,size = np.shape(particles.x[len(particles.x)//2:]))
# particles.x = np.array([[-1,0],[1,0]])
# particles.v = np.random.normal(size = np.shape(particles.x))



In [ ]:
particles.get_rho()
particles.plot_particles(density=True)

In [ ]:
particles.plot_potential()

In [ ]:
make_animation(particles,50,dt = 1)

In [ ]:
particles = Particles(npart = 2,n=100,periodic = False)
particles.soft = 2
particles.x = np.array([[35,50.],[60,50.]])
particles.v = np.array([[0.,0.12],[0.,-0.12]])
# particles.IC_Gauss(4,[[-1,-1],[-1,1],[1,-1],[1,1]],[0.25,0.25,0.25,0.25])
# particles.convert_to_gridcoords()

particles.get_rho()
particles.plot_particles(density = True)
particles.plot_potential()

In [ ]:
make_animation(particles,200,dt = 2)

In [12]:
particles = Particles(npart = 100000,n=250,periodic = True)
particles.IC_box()
particles.convert_to_gridcoords()
particles.get_rho()
particles.plot_particles(density = True)
particles.v = np.random.uniform(-30,30,np.shape(particles.v))

In [ ]:
make_animation(particles,300,dt = 0.05,solver = 'rk4')